# Join large date sets using Spark 

This notebook is a part of the project Library recomender system. To generet the datases used in this notebook, we have used spiders to colect biblographis registers and users trasactions over  many  library sites.
The bibliograthic registers have been already unificated and associated with a unique identifier class "codObra";
Fig 1 shows this process 



Ever time that the spider find new trasanctios, its necessary to convert the local id to the "codObra" id. This notebook is about to join tow dataset and create other as result.
The first dataset to be use "reg.txt  corelate  for each local id to codObra,  second data set containd new transactios with locals ids. 
The results is a new dataser with new transactios with one with the coraleted "codObra"



In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("spark joinf table ") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.memory.fraction", 0.8) \
    .config("spark.executor.memory", "14g") \
    .config("spark.driver.memory", "12g")\
    .config("spark.sql.shuffle.partitions" , "8000") \
    .getOrCreate()




In [4]:
path= '/home/JobResults/arrumatransacional/trsOld.csv'
path= '/home/JobResults/arrumatransacional/trs.csv'
transacional = spark.read.load(path, format="csv", header="true", sep=',')


                    
# Displays the content of the DataFrame to stdout
transacional.registerTempTable("transacional")
transacional.show(2)



+------+------+-----+----------+
|codtit|codusu|fonte|    titulo|
+------+------+-----+----------+
|     4|292421|Skoob|Fernão_Cap|
|     4|270521|Skoob|Fernão_Cap|
+------+------+-----+----------+
only showing top 2 rows



In [14]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import lower, col

sc = spark.sparkContext

sqlContext = SQLContext(sc)
sql = "SELECt fonte, count(*) from  transacional group by fonte  "
#sql = "SELECt lower(fonte) from transacional where codtit = 1735  "

result =  sqlContext.sql(sql)
result.show(13)

+----------+--------+
|     fonte|count(1)|
+----------+--------+
|    IDG-RJ| 1486913|
|Amorim_Lim|   57201|
|     Skoob|19569713|
|    Apoesp|    7527|
|       smc| 7536946|
|Colorado_U|   81136|
+----------+--------+



In [8]:
result =  sqlContext.sql("SELECt count(*) from  transacional   ")

result.show(5) # 38 (28) M !!

+--------+
|count(1)|
+--------+
|28739436|
+--------+



In [9]:
# spark is an existing SparkSession
path= '/home/JobResults/arrumatransacional/reg.cvs'

registros = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(path)
                    
# Displays the content of the DataFrame to stdout
registros.registerTempTable("registros")
registros.show(2)

# _c0 codObra
# _c1 codtit 
# _c2 fonte  

+---+-----+-------+
|_c0|  _c1|    _c2|
+---+-----+-------+
|101| 4591|apeoesp|
|101|11266|    SMC|
+---+-----+-------+
only showing top 2 rows



In [20]:
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)
sql ="SELECT   distinct _c2     from  registros "
sql ="SELECT   *     from  registros  where _c0 = 1735  and _c2 like 'A%'"

print(sql)
result =  sqlContext.sql(sql)
result.show(2)




SELECT   *     from  registros  where _c0 = 1735  and _c2 like 'A%'
+----+----+-------------+
| _c0| _c1|          _c2|
+----+----+-------------+
|1735|1407|Amigos do Bem|
|1735|1544|Amigos do Bem|
+----+----+-------------+
only showing top 2 rows



In [16]:


from pyspark.sql import SQLContext

sc = spark.sparkContext

sqlContext = SQLContext(sc)

sql = ' SELECT transacional.codusu , _c0 as codobra , titulo, transacional.fonte, 5 as rating '
sql  = sql  + '  FROM transacional '
sql  = sql  +  ' INNER JOIN  registros  ON  concat(string( registros._c1), lower(registros._c2))  =  '
sql  = sql  +  " concat(string( transacional.codtit), lower(transacional.fonte))  "


print(sql)

result =  sqlContext.sql(sql)
result.show(3)

result.registerTempTable("transacoesObras")
result.toPandas().to_csv('transacoesObras.csv')




 SELECT transacional.codusu , _c0 as codobra , titulo, transacional.fonte, 5 as rating   FROM transacional  INNER JOIN  registros  ON  concat(string( registros._c1), lower(registros._c2))  =   concat(string( transacional.codtit), lower(transacional.fonte)) and transacional.fonte = 'Apoesp' 
+------+-------+------+-----+------+
|codusu|codobra|titulo|fonte|rating|
+------+-------+------+-----+------+
+------+-------+------+-----+------+



In [14]:

transactiosR =  sqlContext.sql("select count(*) from  transacoesObras " )
transactiosR.show(4) # 16M

+--------+
|count(1)|
+--------+
|16500598|
+--------+



In [21]:
from pyspark.sql import SQLContext

sc = spark.sparkContext

sqlContext = SQLContext(sc)
transactiosR =  sqlContext.sql("select * from  transacoesObras ")
transactiosR.show(4)

+------+-------+------+-----+------+
|codusu|codobra|titulo|fonte|rating|
+------+-------+------+-----+------+
+------+-------+------+-----+------+



In [ ]:
spark.stop()